<a href="https://colab.research.google.com/github/davidlealo/usando_hf/blob/main/modelo_coherence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de Librerías Necesarias

A continuación, se instalan las librerías esenciales para este proyecto:

1. **Transformers**: Facilita el uso de modelos de lenguaje preentrenados de Hugging Face.
2. **Accelerate**: Optimiza el despliegue y la ejecución de modelos en múltiples dispositivos.
3. **Safetensors**: Carga y almacena modelos de manera segura y eficiente.
4. **PyPDF2**: Permite extraer texto de archivos PDF.
5. **Python-Dotenv**: Maneja variables de entorno de forma segura.
6. **PyCryptodome**: Requerida para manejar archivos cifrados o protegidos, como algunos PDFs.

Cada librería tiene un rol específico, desde manipular datos hasta optimizar modelos de lenguaje.

---


In [9]:
# Instalar la última versión de Transformers desde el repositorio oficial
!pip install 'git+https://github.com/huggingface/transformers.git' --upgrade
!pip install accelerate safetensors
!pip install PyPDF2 transformers python-dotenv
!pip install pycryptodome

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-nx7mtlup
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-nx7mtlup
  Resolved https://github.com/huggingface/transformers.git to commit 0531d7513b617f7c5f8b5f333985c63f0edd5fe2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.3 MB/s eta 0:00:00


# Carga el Token en tu Script

In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener el token de Hugging Face
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

# Iniciar sesión en Hugging Face
login(token=huggingface_token)
print("Autenticación exitosa con Hugging Face.")


Autenticación exitosa con Hugging Face.


# Cargar el Modelo y Tokenizer

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Nombre del modelo
model_id = "CohereForAI/c4ai-command-r7b-12-2024"

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Usar BF16 para eficiencia
    device_map="auto"  # Distribuye automáticamente en GPU si está disponible
)

# Confirmar que el modelo está cargado
print("Modelo cargado correctamente.")


tokenizer_config.json:   0%|          | 0.00/44.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/178 [00:00<?, ?B/s]

Modelo cargado correctamente.


# Generar Texto con el Modelo

Conversación con el Modelo en Modo Conversacional

In [5]:
# Crear una plantilla de chat
messages = [{"role": "user", "content": "Hola, ¿cuáles son las últimas tendencias en IA?"}]

# Aplicar el formato de chat
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

# Generar la respuesta
gen_tokens = model.generate(
    input_ids,
    max_new_tokens=200,  # Longitud máxima de la respuesta
    do_sample=True,      # Sampling para respuestas variadas
    temperature=0.7,     # Controlar la creatividad de la respuesta
)

# Decodificar y mostrar la salida
output = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
print("Respuesta del modelo:")
print(output)


The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Respuesta del modelo:
<|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Hola, ¿cuáles son las últimas tendencias en IA?<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>La inteligencia artificial (IA) está experimentando un rápido desarrollo y evolución, con nuevas tendencias y avances que están transformando diversos sectores. A continuación, se presentan algunas de las últimas tendencias en el campo de la IA:

1. Aprendizaje profundo (Deep Learning): El aprendizaje profundo es una rama del aprendizaje automático que se basa en redes neuronales artificiales con múltiples capas. Esta técnica ha demostrado ser muy eficaz en áreas como el reconocimiento de imágenes, el procesamiento del lenguaje natural, la traducción automática y el análisis de datos complejos. Los modelos de redes neuronales convolucionales (CNN) y las redes neuronales recurrentes (RNN) son populares en este contexto.

2. Aprendizaje automático (Machine Learning): El aprendizaje automático es una rama d

# Trabajar con RAG

Proceso para Usar Documentos en RAG
1. Preparar tus documentos
Convierte tus documentos a texto plano si aún no lo están (por ejemplo, desde PDF, Word, etc.).

2. Cargar y dividir los documentos
Divide el texto en fragmentos más pequeños que puedan ser manejados como contexto. Esto se hace porque los modelos de lenguaje tienen límites de longitud (en este caso, 128K tokens para el modelo C4AI Command R7B).

3. Integrar los fragmentos como contexto en el modelo
Usa un formato compatible, como la plantilla de chat del modelo, para agregar estos fragmentos como entradas.




### Carga los PDFs desde URLs

In [8]:
import requests
from PyPDF2 import PdfReader
import os

# Lista de URLs con descripciones corregidas
documents = [
    {"url": "https://www.curriculumnacional.cl/614/articles-354717_bases.pdf",
     "descripcion": "Bases Curriculares para la Educación de Personas Jóvenes y Adultas - 2024"},
    {"url": "https://www.curriculumnacional.cl/614/articles-22394_bases.pdf",
     "descripcion": "Bases Curriculares Primero a Sexto Básico - Ministerio de Educación 2012"},
    {"url": "https://www.curriculumnacional.cl/614/articles-143635_bases.pdf",
     "descripcion": "BASES CURRICULARES LENGUA Y CULTURA DE LOS PUEBLOS ORIGINARIOS ANCESTRALES 1° a 6° año de Educación Básica"},
    {"url": "https://www.curriculumnacional.cl/614/articles-37136_bases.pdf",
     "descripcion": "Bases Curriculares 7º básico a 2º medio - Ministerio de Educación 2015"},
    {"url": "https://www.curriculumnacional.cl/614/articles-91414_bases.pdf",
     "descripcion": "Bases Curriculares 3º y 4º medio - Ministerio de Educación 2019"},
    {"url": "https://www.curriculumnacional.cl/614/articles-69957_bases.pdf",
     "descripcion": "BasesCurriculares Educación Parvularia"},
    {"url": "https://www.curriculumnacional.cl/614/articles-70892_bases.pdf",
     "descripcion": "Bases Curriculares Formación Diferenciada Técnico-Profesional Especialidades y Perfiles de Egreso - Ministerio de Educación 2016"},
]

# Carpeta para guardar los PDFs
pdf_folder = "curriculum_pdfs"
os.makedirs(pdf_folder, exist_ok=True)

# Descargar y guardar los PDFs con nombres basados en la descripción
pdf_paths = []
for doc in documents:
    response = requests.get(doc["url"])
    file_name = doc["descripcion"].replace(" ", "_").replace("°", "").replace("(", "").replace(")", "") + ".pdf"  # Crear nombre de archivo único
    pdf_path = os.path.join(pdf_folder, file_name)
    with open(pdf_path, "wb") as file:
        file.write(response.content)
    pdf_paths.append({"path": pdf_path, "descripcion": doc["descripcion"]})

# Imprimir resumen de descarga
for pdf in pdf_paths:
    print(f"Descargado: {pdf['descripcion']} -> {pdf['path']}")

print(f"\nSe descargaron {len(pdf_paths)} documentos PDF.")


Descargado: Bases Curriculares para la Educación de Personas Jóvenes y Adultas - 2024 -> curriculum_pdfs/Bases_Curriculares_para_la_Educación_de_Personas_Jóvenes_y_Adultas_-_2024.pdf
Descargado: Bases Curriculares Primero a Sexto Básico - Ministerio de Educación 2012 -> curriculum_pdfs/Bases_Curriculares_Primero_a_Sexto_Básico_-_Ministerio_de_Educación_2012.pdf
Descargado: BASES CURRICULARES LENGUA Y CULTURA DE LOS PUEBLOS ORIGINARIOS ANCESTRALES 1° a 6° año de Educación Básica -> curriculum_pdfs/BASES_CURRICULARES_LENGUA_Y_CULTURA_DE_LOS_PUEBLOS_ORIGINARIOS_ANCESTRALES_1_a_6_año_de_Educación_Básica.pdf
Descargado: Bases Curriculares 7º básico a 2º medio - Ministerio de Educación 2015 -> curriculum_pdfs/Bases_Curriculares_7º_básico_a_2º_medio_-_Ministerio_de_Educación_2015.pdf
Descargado: Bases Curriculares 3º y 4º medio - Ministerio de Educación 2019 -> curriculum_pdfs/Bases_Curriculares_3º_y_4º_medio_-_Ministerio_de_Educación_2019.pdf
Descargado: BasesCurriculares Educación Parvulari

### Extraer Texto de los PDFs
Con los archivos PDF descargados, extraemos el texto usando PyPDF2:


In [11]:
# Función para extraer texto de PDFs
def extract_text_from_pdfs(pdf_paths):
    all_text = ""
    for pdf_info in pdf_paths:
        path = pdf_info["path"]  # Extraer la ruta del archivo
        try:
            reader = PdfReader(path)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:  # Evitar páginas vacías
                    all_text += page_text + "\n"
        except PdfReadError as e:
            print(f"Error leyendo el archivo {path}: {e}")
        except Exception as e:
            print(f"Otro error con el archivo {pdf_info}: {e}")
    return all_text

# Extraer el texto de los documentos
text = extract_text_from_pdfs(pdf_paths)

# Guardar el texto extraído en un archivo
with open("extracted_text.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(text)

print("Se ha extraído el texto de los documentos.")


Otro error con el archivo {'path': 'curriculum_pdfs/Bases_Curriculares_para_la_Educación_de_Personas_Jóvenes_y_Adultas_-_2024.pdf', 'descripcion': 'Bases Curriculares para la Educación de Personas Jóvenes y Adultas - 2024'}: PyCryptodome is required for AES algorithm
Se ha extraído el texto de los documentos.


### Dividir el Texto en Fragmentos
Divide el texto extraído en fragmentos manejables para el modelo.


In [12]:
from transformers import AutoTokenizer

# Cargar el tokenizer del modelo
tokenizer = AutoTokenizer.from_pretrained("CohereForAI/c4ai-command-r7b-12-2024")

# Función para dividir texto en fragmentos de tokens
def chunk_text(text, chunk_size=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return [tokens[i:i+chunk_size] for i in range(0, len(tokens), chunk_size)]

# Dividir el texto en fragmentos
chunks = chunk_text(text)
print(f"Se crearon {len(chunks)} fragmentos de texto.")


Se crearon 1196 fragmentos de texto.


### Utilizar Fragmentos como Contexto en el Modelo
Ahora puedes integrar los fragmentos relevantes al modelo en formato RAG:

In [ ]:
from transformers import AutoModelForCausalLM
import torch

# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained(
    "CohereForAI/c4ai-command-r7b-12-2024",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Consulta del usuario
query = "¿Qué información importante sobre el currículum nacional está en estos documentos?"

# Usar algunos fragmentos como contexto
context_snippets = tokenizer.decode(chunks[0])  # Tomar un fragmento como ejemplo

# Ajustar los mensajes con roles alternados
messages = [
    {"role": "user", "content": "Por favor utiliza el siguiente contexto para responder mi pregunta."},
    {"role": "assistant", "content": f"Contexto proporcionado: {context_snippets}"},
    {"role": "user", "content": query}
]

# Aplicar formato de chat
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

# Generar respuesta
output_ids = model.generate(input_ids, max_new_tokens=200, temperature=0.7)
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Respuesta del modelo:")
print(response)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2135: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(
